In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import csv
from sklearn.mixture import GaussianMixture
import librosa
import librosa.display
import IPython.display as ipd
from glob import glob
from sklearn.cluster import KMeans
from librosa import feature
from scipy.stats import skew, kurtosis
from sklearn.cluster import AgglomerativeClustering
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

#path of the directories seperated by their genres
data_dir = r'C:\\20+\\araproje\\deneme'
print(list(os.listdir(data_dir))) #to see the directories inside

#directories that includes wav files
dir_list = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

['metal.00000_1.wav', 'metal.00000_2.wav', 'metal.00000_3.wav', 'metal.00000_4.wav', 'metal.00000_5.wav', 'metal.00001_1.wav', 'metal.00001_2.wav', 'metal.00001_3.wav', 'metal.00001_4.wav', 'metal.00001_5.wav', 'zart', 'zort']


In [2]:
def calcStatVals(vector):
    '''
    function that returns specific statistical values of given feature vector
    :param vector: feature vector
    :return: list of statistical values of given vector
    '''
    returnVec = []
    try:
        returnVec.extend((np.mean(vector),
        np.std(vector),
        np.amin(vector),
        np.amax(vector),
        np.median(vector),
        skew(vector.flatten()),
        kurtosis(vector.flatten())))
    except ValueError:
        return [0,0,0,0,0,0,0]

    return returnVec

def extractFeature(y, algorithm):
    '''
    Calls wanted clustering algorithm for each features of given audio time series
    and returns them as an appended list
    :param y: audio time series
    :param algorithm: wanted clustering algorithm
    :return: list: appended clustered features
    '''
    returnVec = []

    #clusterTo2 can be changed to clusterTo3 for purposing to cluster 3
    returnVec += clusterTo2(feature.chroma_stft(y), algorithm) + \
                 clusterTo2(feature.chroma_cqt(y), algorithm) + \
                 clusterTo2(feature.spectral_centroid(y), algorithm) + \
                 clusterTo2(feature.spectral_bandwidth(y), algorithm) + \
                 clusterTo2(feature.spectral_rolloff(y), algorithm) + \
                 clusterTo2(feature.spectral_contrast(y), algorithm)

    #split mfcc to 20 mfc
    mfcc = feature.mfcc(y, n_mfcc=20)
    for mfccI in mfcc:
        returnVec += clusterTo2(mfccI, algorithm)

    return returnVec

#headers for no cluster file. better to shrunk the block
header_1 = [
    'chroma_stft_1_mean', 'chroma_stft_1_std_dev', 'chroma_stft_1_min', 'chroma_stft_1_max', 'chroma_stft_1_median',
    'chroma_stft_1_skew', 'chroma_stft_1_kurtosis',

    'chroma_cqt_1_mean', 'chroma_cqt_1_std_dev', 'chroma_cqt_1_min', 'chroma_cqt_1_max', 'chroma_cqt_1_median',
    'chroma_cqt_1_skew', 'chroma_cqt_1_kurtosis',

    'spectral_centroid_1_mean', 'spectral_centroid_1_std_dev', 'spectral_centroid_1_min', 'spectral_centroid_1_max',
    'spectral_centroid_1_median', 'spectral_centroid_1_skew', 'spectral_centroid_1_kurtosis',

    'spectral_bandwidth_1_mean', 'spectral_bandwidth_1_std_dev', 'spectral_bandwidth_1_min', 'spectral_bandwidth_1_max',
    'spectral_bandwidth_1_median', 'spectral_bandwidth_1_skew', 'spectral_bandwidth_1_kurtosis',

    'spectral_rolloff_1_mean', 'spectral_rolloff_1_std_dev', 'spectral_rolloff_1_min', 'spectral_rolloff_1_max',
    'spectral_rolloff_1_median', 'spectral_rolloff_1_skew', 'spectral_rolloff_1_kurtosis',

    'spectral_contrast_1_mean', 'spectral_contrast_1_std_dev', 'spectral_contrast_1_min', 'spectral_contrast_1_max',
    'spectral_contrast_1_median', 'spectral_contrast_1_skew', 'spectral_contrast_1_kurtosis',

    'mfcc_1_1_mean', 'mfcc_1_1_std_dev', 'mfcc_1_1_min', 'mfcc_1_1_max',
    'mfcc_1_1_median', 'mfcc_1_1_skew', 'mfcc_1_1_kurtosis',

    'mfcc_2_1_mean', 'mfcc_2_1_std_dev', 'mfcc_2_1_min', 'mfcc_2_1_max',
    'mfcc_2_1_median', 'mfcc_2_1_skew', 'mfcc_2_1_kurtosis',

    'mfcc_3_1_mean', 'mfcc_3_1_std_dev', 'mfcc_3_1_min', 'mfcc_3_1_max',
    'mfcc_3_1_median', 'mfcc_3_1_skew', 'mfcc_3_1_kurtosis',

    'mfcc_4_1_mean', 'mfcc_4_1_std_dev', 'mfcc_4_1_min', 'mfcc_4_1_max',
    'mfcc_4_1_median', 'mfcc_4_1_skew', 'mfcc_4_1_kurtosis',

    'mfcc_5_1_mean', 'mfcc_5_1_std_dev', 'mfcc_5_1_min', 'mfcc_5_1_max',
    'mfcc_5_1_median', 'mfcc_5_1_skew', 'mfcc_5_1_kurtosis',

    'mfcc_6_1_mean', 'mfcc_6_1_std_dev', 'mfcc_6_1_min', 'mfcc_6_1_max',
    'mfcc_6_1_median', 'mfcc_6_1_skew', 'mfcc_6_1_kurtosis',

    'mfcc_7_1_mean', 'mfcc_7_1_std_dev', 'mfcc_7_1_min', 'mfcc_7_1_max',
    'mfcc_7_1_median', 'mfcc_7_1_skew', 'mfcc_7_1_kurtosis',

    'mfcc_8_1_mean', 'mfcc_8_1_std_dev', 'mfcc_8_1_min', 'mfcc_8_1_max',
    'mfcc_8_1_median', 'mfcc_8_1_skew', 'mfcc_8_1_kurtosis',

    'mfcc_9_1_mean', 'mfcc_9_1_std_dev', 'mfcc_9_1_min', 'mfcc_9_1_max',
    'mfcc_9_1_median', 'mfcc_9_1_skew', 'mfcc_9_1_kurtosis',

    'mfcc_10_1_mean', 'mfcc_10_1_std_dev', 'mfcc_10_1_min', 'mfcc_10_1_max',
    'mfcc_10_1_median', 'mfcc_10_1_skew', 'mfcc_10_1_kurtosis',

    'mfcc_11_1_mean', 'mfcc_11_1_std_dev', 'mfcc_11_1_min', 'mfcc_11_1_max',
    'mfcc_11_1_median', 'mfcc_11_1_skew', 'mfcc_11_1_kurtosis',

    'mfcc_12_1_mean', 'mfcc_12_1_std_dev', 'mfcc_12_1_min', 'mfcc_12_1_max',
    'mfcc_12_1_median', 'mfcc_12_1_skew', 'mfcc_12_1_kurtosis',

    'mfcc_13_1_mean', 'mfcc_13_1_std_dev', 'mfcc_13_1_min', 'mfcc_13_1_max',
    'mfcc_13_1_median', 'mfcc_13_1_skew', 'mfcc_13_1_kurtosis',

    'mfcc_14_1_mean', 'mfcc_14_1_std_dev', 'mfcc_14_1_min', 'mfcc_14_1_max',
    'mfcc_14_1_median', 'mfcc_14_1_skew', 'mfcc_14_1_kurtosis',


    'mfcc_15_1_mean', 'mfcc_15_1_std_dev', 'mfcc_15_1_min', 'mfcc_15_1_max',
    'mfcc_15_1_median', 'mfcc_15_1_skew', 'mfcc_15_1_kurtosis',

    'mfcc_16_1_mean', 'mfcc_16_1_std_dev', 'mfcc_16_1_min', 'mfcc_16_1_max',
    'mfcc_16_1_median', 'mfcc_16_1_skew', 'mfcc_16_1_kurtosis',

    'mfcc_17_1_mean', 'mfcc_17_1_std_dev', 'mfcc_17_1_min', 'mfcc_17_1_max',
    'mfcc_17_1_median', 'mfcc_17_1_skew', 'mfcc_17_1_kurtosis',

    'mfcc_18_1_mean', 'mfcc_18_1_std_dev', 'mfcc_18_1_min', 'mfcc_18_1_max',
    'mfcc_18_1_median', 'mfcc_18_1_skew', 'mfcc_18_1_kurtosis',

    'mfcc_19_1_mean', 'mfcc_19_1_std_dev', 'mfcc_19_1_min', 'mfcc_19_1_max',
    'mfcc_19_1_median', 'mfcc_19_1_skew', 'mfcc_19_1_kurtosis',

    'mfcc_20_1_mean', 'mfcc_20_1_std_dev', 'mfcc_20_1_min', 'mfcc_20_1_max',
    'mfcc_20_1_median', 'mfcc_20_1_skew', 'mfcc_20_1_kurtosis',
    "label"
]

#headers for 2 clusters. better to shrunk the block
header_2 = [
    'chroma_stft_1_mean', 'chroma_stft_1_std_dev', 'chroma_stft_1_min', 'chroma_stft_1_max', 'chroma_stft_1_median',
    'chroma_stft_1_skew', 'chroma_stft_1_kurtosis',
    'chroma_stft_2_mean', 'chroma_stft_2_std_dev', 'chroma_stft_2_min', 'chroma_stft_2_max', 'chroma_stft_2_median',
    'chroma_stft_2_skew', 'chroma_stft_2_kurtosis',

    'chroma_cqt_1_mean', 'chroma_cqt_1_std_dev', 'chroma_cqt_1_min', 'chroma_cqt_1_max', 'chroma_cqt_1_median',
    'chroma_cqt_1_skew', 'chroma_cqt_1_kurtosis',
    'chroma_cqt_2_mean', 'chroma_cqt_2_std_dev', 'chroma_cqt_2_min', 'chroma_cqt_2_max', 'chroma_cqt_2_median',
    'chroma_cqt_2_skew', 'chroma_cqt_2_kurtosis',

    'spectral_centroid_1_mean', 'spectral_centroid_1_std_dev', 'spectral_centroid_1_min', 'spectral_centroid_1_max',
    'spectral_centroid_1_median', 'spectral_centroid_1_skew', 'spectral_centroid_1_kurtosis',
    'spectral_centroid_2_mean', 'spectral_centroid_2_std_dev', 'spectral_centroid_2_min', 'spectral_centroid_2_max',
    'spectral_centroid_2_median', 'spectral_centroid_2_skew', 'spectral_centroid_2_kurtosis',

    'spectral_bandwidth_1_mean', 'spectral_bandwidth_1_std_dev', 'spectral_bandwidth_1_min', 'spectral_bandwidth_1_max',
    'spectral_bandwidth_1_median', 'spectral_bandwidth_1_skew', 'spectral_bandwidth_1_kurtosis',
    'spectral_bandwidth_2_mean', 'spectral_bandwidth_2_std_dev', 'spectral_bandwidth_2_min', 'spectral_bandwidth_2_max',
    'spectral_bandwidth_2_median', 'spectral_bandwidth_2_skew', 'spectral_bandwidth_2_kurtosis',

    'spectral_rolloff_1_mean', 'spectral_rolloff_1_std_dev', 'spectral_rolloff_1_min', 'spectral_rolloff_1_max',
    'spectral_rolloff_1_median', 'spectral_rolloff_1_skew', 'spectral_rolloff_1_kurtosis',
    'spectral_rolloff_2_mean', 'spectral_rolloff_2_std_dev', 'spectral_rolloff_2_min', 'spectral_rolloff_2_max',
    'spectral_rolloff_2_median', 'spectral_rolloff_2_skew', 'spectral_rolloff_2_kurtosis',

    'spectral_contrast_1_mean', 'spectral_contrast_1_std_dev', 'spectral_contrast_1_min', 'spectral_contrast_1_max',
    'spectral_contrast_1_median', 'spectral_contrast_1_skew', 'spectral_contrast_1_kurtosis',
    'spectral_contrast_2_mean', 'spectral_contrast_2_std_dev', 'spectral_contrast_2_min', 'spectral_contrast_2_max',
    'spectral_contrast_2_median', 'spectral_contrast_2_skew', 'spectral_contrast_2_kurtosis',

    'mfcc_1_1_mean', 'mfcc_1_1_std_dev', 'mfcc_1_1_min', 'mfcc_1_1_max',
    'mfcc_1_1_median', 'mfcc_1_1_skew', 'mfcc_1_1_kurtosis',
    'mfcc_1_2_mean', 'mfcc_1_2_std_dev', 'mfcc_1_2_min', 'mfcc_1_2_max',
    'mfcc_1_2_median', 'mfcc_1_2_skew', 'mfcc_1_2_kurtosis',

    'mfcc_2_1_mean', 'mfcc_2_1_std_dev', 'mfcc_2_1_min', 'mfcc_2_1_max',
    'mfcc_2_1_median', 'mfcc_2_1_skew', 'mfcc_2_1_kurtosis',
    'mfcc_2_2_mean', 'mfcc_2_2_std_dev', 'mfcc_2_2_min', 'mfcc_2_2_max',
    'mfcc_2_2_median', 'mfcc_2_2_skew', 'mfcc_2_2_kurtosis',

    'mfcc_3_1_mean', 'mfcc_3_1_std_dev', 'mfcc_3_1_min', 'mfcc_3_1_max',
    'mfcc_3_1_median', 'mfcc_3_1_skew', 'mfcc_3_1_kurtosis',
    'mfcc_3_2_mean', 'mfcc_3_2_std_dev', 'mfcc_3_2_min', 'mfcc_3_2_max',
    'mfcc_3_2_median', 'mfcc_3_2_skew', 'mfcc_3_2_kurtosis',

    'mfcc_4_1_mean', 'mfcc_4_1_std_dev', 'mfcc_4_1_min', 'mfcc_4_1_max',
    'mfcc_4_1_median', 'mfcc_4_1_skew', 'mfcc_4_1_kurtosis',
    'mfcc_4_2_mean', 'mfcc_4_2_std_dev', 'mfcc_4_2_min', 'mfcc_4_2_max',
    'mfcc_4_2_median', 'mfcc_4_2_skew', 'mfcc_4_2_kurtosis',

    'mfcc_5_1_mean', 'mfcc_5_1_std_dev', 'mfcc_5_1_min', 'mfcc_5_1_max',
    'mfcc_5_1_median', 'mfcc_5_1_skew', 'mfcc_5_1_kurtosis',
    'mfcc_5_2_mean', 'mfcc_5_2_std_dev', 'mfcc_5_2_min', 'mfcc_5_2_max',
    'mfcc_5_2_median', 'mfcc_5_2_skew', 'mfcc_5_2_kurtosis',

    'mfcc_6_1_mean', 'mfcc_6_1_std_dev', 'mfcc_6_1_min', 'mfcc_6_1_max',
    'mfcc_6_1_median', 'mfcc_6_1_skew', 'mfcc_6_1_kurtosis',
    'mfcc_6_2_mean', 'mfcc_6_2_std_dev', 'mfcc_6_2_min', 'mfcc_6_2_max',
    'mfcc_6_2_median', 'mfcc_6_2_skew', 'mfcc_6_2_kurtosis',

    'mfcc_7_1_mean', 'mfcc_7_1_std_dev', 'mfcc_7_1_min', 'mfcc_7_1_max',
    'mfcc_7_1_median', 'mfcc_7_1_skew', 'mfcc_7_1_kurtosis',
    'mfcc_7_2_mean', 'mfcc_7_2_std_dev', 'mfcc_7_2_min', 'mfcc_7_2_max',
    'mfcc_7_2_median', 'mfcc_7_2_skew', 'mfcc_7_2_kurtosis',

    'mfcc_8_1_mean', 'mfcc_8_1_std_dev', 'mfcc_8_1_min', 'mfcc_8_1_max',
    'mfcc_8_1_median', 'mfcc_8_1_skew', 'mfcc_8_1_kurtosis',
    'mfcc_8_2_mean', 'mfcc_8_2_std_dev', 'mfcc_8_2_min', 'mfcc_8_2_max',
    'mfcc_8_2_median', 'mfcc_8_2_skew', 'mfcc_8_2_kurtosis',

    'mfcc_9_1_mean', 'mfcc_9_1_std_dev', 'mfcc_9_1_min', 'mfcc_9_1_max',
    'mfcc_9_1_median', 'mfcc_9_1_skew', 'mfcc_9_1_kurtosis',
    'mfcc_9_2_mean', 'mfcc_9_2_std_dev', 'mfcc_9_2_min', 'mfcc_9_2_max',
    'mfcc_9_2_median', 'mfcc_9_2_skew', 'mfcc_9_2_kurtosis',

    'mfcc_10_1_mean', 'mfcc_10_1_std_dev', 'mfcc_10_1_min', 'mfcc_10_1_max',
    'mfcc_10_1_median', 'mfcc_10_1_skew', 'mfcc_10_1_kurtosis',
    'mfcc_10_2_mean', 'mfcc_10_2_std_dev', 'mfcc_10_2_min', 'mfcc_10_2_max',
    'mfcc_10_2_median', 'mfcc_10_2_skew', 'mfcc_10_2_kurtosis',

    'mfcc_11_1_mean', 'mfcc_11_1_std_dev', 'mfcc_11_1_min', 'mfcc_11_1_max',
    'mfcc_11_1_median', 'mfcc_11_1_skew', 'mfcc_11_1_kurtosis',
    'mfcc_11_2_mean', 'mfcc_11_2_std_dev', 'mfcc_11_2_min', 'mfcc_11_2_max',
    'mfcc_11_2_median', 'mfcc_11_2_skew', 'mfcc_11_2_kurtosis',

    'mfcc_12_1_mean', 'mfcc_12_1_std_dev', 'mfcc_12_1_min', 'mfcc_12_1_max',
    'mfcc_12_1_median', 'mfcc_12_1_skew', 'mfcc_12_1_kurtosis',
    'mfcc_12_2_mean', 'mfcc_12_2_std_dev', 'mfcc_12_2_min', 'mfcc_12_2_max',
    'mfcc_12_2_median', 'mfcc_12_2_skew', 'mfcc_12_2_kurtosis',

    'mfcc_13_1_mean', 'mfcc_13_1_std_dev', 'mfcc_13_1_min', 'mfcc_13_1_max',
    'mfcc_13_1_median', 'mfcc_13_1_skew', 'mfcc_13_1_kurtosis',
    'mfcc_13_2_mean', 'mfcc_13_2_std_dev', 'mfcc_13_2_min', 'mfcc_13_2_max',
    'mfcc_13_2_median', 'mfcc_13_2_skew', 'mfcc_13_2_kurtosis',

    'mfcc_14_1_mean', 'mfcc_14_1_std_dev', 'mfcc_14_1_min', 'mfcc_14_1_max',
    'mfcc_14_1_median', 'mfcc_14_1_skew', 'mfcc_14_1_kurtosis',
    'mfcc_14_2_mean', 'mfcc_14_2_std_dev', 'mfcc_14_2_min', 'mfcc_14_2_max',
    'mfcc_14_2_median', 'mfcc_14_2_skew', 'mfcc_14_2_kurtosis',

    'mfcc_15_1_mean', 'mfcc_15_1_std_dev', 'mfcc_15_1_min', 'mfcc_15_1_max',
    'mfcc_15_1_median', 'mfcc_15_1_skew', 'mfcc_15_1_kurtosis',
    'mfcc_15_2_mean', 'mfcc_15_2_std_dev', 'mfcc_15_2_min', 'mfcc_15_2_max',
    'mfcc_15_2_median', 'mfcc_15_2_skew', 'mfcc_15_2_kurtosis',

    'mfcc_16_1_mean', 'mfcc_16_1_std_dev', 'mfcc_16_1_min', 'mfcc_16_1_max',
    'mfcc_16_1_median', 'mfcc_16_1_skew', 'mfcc_16_1_kurtosis',
    'mfcc_16_2_mean', 'mfcc_16_2_std_dev', 'mfcc_16_2_min', 'mfcc_16_2_max',
    'mfcc_16_2_median', 'mfcc_16_2_skew', 'mfcc_16_2_kurtosis',

    'mfcc_17_1_mean', 'mfcc_17_1_std_dev', 'mfcc_17_1_min', 'mfcc_17_1_max',
    'mfcc_17_1_median', 'mfcc_17_1_skew', 'mfcc_17_1_kurtosis',
    'mfcc_17_2_mean', 'mfcc_17_2_std_dev', 'mfcc_17_2_min', 'mfcc_17_2_max',
    'mfcc_17_2_median', 'mfcc_17_2_skew', 'mfcc_17_2_kurtosis',

    'mfcc_18_1_mean', 'mfcc_18_1_std_dev', 'mfcc_18_1_min', 'mfcc_18_1_max',
    'mfcc_18_1_median', 'mfcc_18_1_skew', 'mfcc_18_1_kurtosis',
    'mfcc_18_2_mean', 'mfcc_18_2_std_dev', 'mfcc_18_2_min', 'mfcc_18_2_max',
    'mfcc_18_2_median', 'mfcc_18_2_skew', 'mfcc_18_2_kurtosis',

    'mfcc_19_1_mean', 'mfcc_19_1_std_dev', 'mfcc_19_1_min', 'mfcc_19_1_max',
    'mfcc_19_1_median', 'mfcc_19_1_skew', 'mfcc_19_1_kurtosis',
    'mfcc_19_2_mean', 'mfcc_19_2_std_dev', 'mfcc_19_2_min', 'mfcc_19_2_max',
    'mfcc_19_2_median', 'mfcc_19_2_skew', 'mfcc_19_2_kurtosis',

    'mfcc_20_1_mean', 'mfcc_20_1_std_dev', 'mfcc_20_1_min', 'mfcc_20_1_max',
    'mfcc_20_1_median', 'mfcc_20_1_skew', 'mfcc_20_1_kurtosis',
    'mfcc_20_2_mean', 'mfcc_20_2_std_dev', 'mfcc_20_2_min', 'mfcc_20_2_max',
    'mfcc_20_2_median', 'mfcc_20_2_skew', 'mfcc_20_2_kurtosis',

    'label'

]

#headers for 3 clusters better to shrunk the block
header_3 = [
    'chroma_stft_1_mean', 'chroma_stft_1_std_dev', 'chroma_stft_1_min', 'chroma_stft_1_max', 'chroma_stft_1_median',
    'chroma_stft_1_skew', 'chroma_stft_1_kurtosis',
    'chroma_stft_2_mean', 'chroma_stft_2_std_dev', 'chroma_stft_2_min', 'chroma_stft_2_max', 'chroma_stft_2_median',
    'chroma_stft_2_skew', 'chroma_stft_2_kurtosis',
    'chroma_stft_3_mean', 'chroma_stft_3_std_dev', 'chroma_stft_3_min', 'chroma_stft_3_max', 'chroma_stft_3_median',
    'chroma_stft_3_skew', 'chroma_stft_3_kurtosis',

    'chroma_cqt_1_mean', 'chroma_cqt_1_std_dev', 'chroma_cqt_1_min', 'chroma_cqt_1_max', 'chroma_cqt_1_median',
    'chroma_cqt_1_skew', 'chroma_cqt_1_kurtosis',
    'chroma_cqt_2_mean', 'chroma_cqt_2_std_dev', 'chroma_cqt_2_min', 'chroma_cqt_2_max', 'chroma_cqt_2_median',
    'chroma_cqt_2_skew', 'chroma_cqt_2_kurtosis',
    'chroma_cqt_3_mean', 'chroma_cqt_3_std_dev', 'chroma_cqt_3_min', 'chroma_cqt_3_max', 'chroma_cqt_3_median',
    'chroma_cqt_3_skew', 'chroma_cqt_3_kurtosis',

    'spectral_centroid_1_mean', 'spectral_centroid_1_std_dev', 'spectral_centroid_1_min', 'spectral_centroid_1_max',
    'spectral_centroid_1_median', 'spectral_centroid_1_skew', 'spectral_centroid_1_kurtosis',
    'spectral_centroid_2_mean', 'spectral_centroid_2_std_dev', 'spectral_centroid_2_min', 'spectral_centroid_2_max',
    'spectral_centroid_2_median', 'spectral_centroid_2_skew', 'spectral_centroid_2_kurtosis',
    'spectral_centroid_3_mean', 'spectral_centroid_3_std_dev', 'spectral_centroid_3_min', 'spectral_centroid_3_max', 'spectral_centroid_3_median',
    'spectral_centroid_3_skew', 'spectral_centroid_3_kurtosis',

    'spectral_bandwidth_1_mean', 'spectral_bandwidth_1_std_dev', 'spectral_bandwidth_1_min', 'spectral_bandwidth_1_max',
    'spectral_bandwidth_1_median', 'spectral_bandwidth_1_skew', 'spectral_bandwidth_1_kurtosis',
    'spectral_bandwidth_2_mean', 'spectral_bandwidth_2_std_dev', 'spectral_bandwidth_2_min', 'spectral_bandwidth_2_max',
    'spectral_bandwidth_2_median', 'spectral_bandwidth_2_skew', 'spectral_bandwidth_2_kurtosis',
    'spectral_bandwidth_3_mean', 'spectral_bandwidth_3_std_dev', 'spectral_bandwidth_3_min', 'spectral_bandwidth_3_max', 'spectral_bandwidth_3_median',
    'spectral_bandwidth_3_skew', 'spectral_bandwidth_3_kurtosis',

    'spectral_rolloff_1_mean', 'spectral_rolloff_1_std_dev', 'spectral_rolloff_1_min', 'spectral_rolloff_1_max',
    'spectral_rolloff_1_median', 'spectral_rolloff_1_skew', 'spectral_rolloff_1_kurtosis',
    'spectral_rolloff_2_mean', 'spectral_rolloff_2_std_dev', 'spectral_rolloff_2_min', 'spectral_rolloff_2_max',
    'spectral_rolloff_2_median', 'spectral_rolloff_2_skew', 'spectral_rolloff_2_kurtosis',
    'spectral_rolloff_3_mean', 'spectral_rolloff_3_std_dev', 'spectral_rolloff_3_min', 'spectral_rolloff_3_max', 'spectral_rolloff_3_median',
    'spectral_rolloff_3_skew', 'spectral_rolloff_3_kurtosis',

    'spectral_contrast_1_mean', 'spectral_contrast_1_std_dev', 'spectral_contrast_1_min', 'spectral_contrast_1_max',
    'spectral_contrast_1_median', 'spectral_contrast_1_skew', 'spectral_contrast_1_kurtosis',
    'spectral_contrast_2_mean', 'spectral_contrast_2_std_dev', 'spectral_contrast_2_min', 'spectral_contrast_2_max',
    'spectral_contrast_2_median', 'spectral_contrast_2_skew', 'spectral_contrast_2_kurtosis',
    'spectral_contrast_3_mean', 'spectral_contrast_3_std_dev', 'spectral_contrast_3_min', 'spectral_contrast_3_max', 'spectral_contrast_3_median',
    'spectral_contrast_3_skew', 'spectral_contrast_3_kurtosis',

    'mfcc_1_1_mean', 'mfcc_1_1_std_dev', 'mfcc_1_1_min', 'mfcc_1_1_max',
    'mfcc_1_1_median', 'mfcc_1_1_skew', 'mfcc_1_1_kurtosis',
    'mfcc_1_2_mean', 'mfcc_1_2_std_dev', 'mfcc_1_2_min', 'mfcc_1_2_max',
    'mfcc_1_2_median', 'mfcc_1_2_skew', 'mfcc_1_2_kurtosis',
    'mfcc_1_3_mean', 'mfcc_1_3_std_dev', 'mfcc_1_3_min', 'mfcc_1_3_max', 'mfcc_1_3_median',
    'mfcc_1_3_skew', 'mfcc_1_3_kurtosis',

    'mfcc_2_1_mean', 'mfcc_2_1_std_dev', 'mfcc_2_1_min', 'mfcc_2_1_max',
    'mfcc_2_1_median', 'mfcc_2_1_skew', 'mfcc_2_1_kurtosis',
    'mfcc_2_2_mean', 'mfcc_2_2_std_dev', 'mfcc_2_2_min', 'mfcc_2_2_max',
    'mfcc_2_2_median', 'mfcc_2_2_skew', 'mfcc_2_2_kurtosis',
    'mfcc_2_3_mean', 'mfcc_2_3_std_dev', 'mfcc_2_3_min', 'mfcc_2_3_max', 'mfcc_2_3_median',
    'mfcc_2_3_skew', 'mfcc_2_3_kurtosis',

    'mfcc_3_1_mean', 'mfcc_3_1_std_dev', 'mfcc_3_1_min', 'mfcc_3_1_max',
    'mfcc_3_1_median', 'mfcc_3_1_skew', 'mfcc_3_1_kurtosis',
    'mfcc_3_2_mean', 'mfcc_3_2_std_dev', 'mfcc_3_2_min', 'mfcc_3_2_max',
    'mfcc_3_2_median', 'mfcc_3_2_skew', 'mfcc_3_2_kurtosis',
    'mfcc_3_3_mean', 'mfcc_3_3_std_dev', 'mfcc_3_3_min', 'mfcc_3_3_max', 'mfcc_3_3_median',
    'mfcc_3_3_skew', 'mfcc_3_3_kurtosis',

    'mfcc_4_1_mean', 'mfcc_4_1_std_dev', 'mfcc_4_1_min', 'mfcc_4_1_max',
    'mfcc_4_1_median', 'mfcc_4_1_skew', 'mfcc_4_1_kurtosis',
    'mfcc_4_2_mean', 'mfcc_4_2_std_dev', 'mfcc_4_2_min', 'mfcc_4_2_max',
    'mfcc_4_2_median', 'mfcc_4_2_skew', 'mfcc_4_2_kurtosis',
    'mfcc_4_3_mean', 'mfcc_4_3_std_dev', 'mfcc_4_3_min', 'mfcc_4_3_max', 'mfcc_4_3_median',
    'mfcc_4_3_skew', 'mfcc_4_3_kurtosis',

    'mfcc_5_1_mean', 'mfcc_5_1_std_dev', 'mfcc_5_1_min', 'mfcc_5_1_max',
    'mfcc_5_1_median', 'mfcc_5_1_skew', 'mfcc_5_1_kurtosis',
    'mfcc_5_2_mean', 'mfcc_5_2_std_dev', 'mfcc_5_2_min', 'mfcc_5_2_max',
    'mfcc_5_2_median', 'mfcc_5_2_skew', 'mfcc_5_2_kurtosis',
    'mfcc_5_3_mean', 'mfcc_5_3_std_dev', 'mfcc_5_3_min', 'mfcc_5_3_max', 'mfcc_5_3_median',
    'mfcc_5_3_skew', 'mfcc_5_3_kurtosis',

    'mfcc_6_1_mean', 'mfcc_6_1_std_dev', 'mfcc_6_1_min', 'mfcc_6_1_max',
    'mfcc_6_1_median', 'mfcc_6_1_skew', 'mfcc_6_1_kurtosis',
    'mfcc_6_2_mean', 'mfcc_6_2_std_dev', 'mfcc_6_2_min', 'mfcc_6_2_max',
    'mfcc_6_2_median', 'mfcc_6_2_skew', 'mfcc_6_2_kurtosis',
    'mfcc_6_3_mean', 'mfcc_6_3_std_dev', 'mfcc_6_3_min', 'mfcc_6_3_max', 'mfcc_6_3_median',
    'mfcc_6_3_skew', 'mfcc_6_3_kurtosis',

    'mfcc_7_1_mean', 'mfcc_7_1_std_dev', 'mfcc_7_1_min', 'mfcc_7_1_max',
    'mfcc_7_1_median', 'mfcc_7_1_skew', 'mfcc_7_1_kurtosis',
    'mfcc_7_2_mean', 'mfcc_7_2_std_dev', 'mfcc_7_2_min', 'mfcc_7_2_max',
    'mfcc_7_2_median', 'mfcc_7_2_skew', 'mfcc_7_2_kurtosis',
    'mfcc_7_3_mean', 'mfcc_7_3_std_dev', 'mfcc_7_3_min', 'mfcc_7_3_max', 'mfcc_7_3_median',
    'mfcc_7_3_skew', 'mfcc_7_3_kurtosis',

    'mfcc_8_1_mean', 'mfcc_8_1_std_dev', 'mfcc_8_1_min', 'mfcc_8_1_max',
    'mfcc_8_1_median', 'mfcc_8_1_skew', 'mfcc_8_1_kurtosis',
    'mfcc_8_2_mean', 'mfcc_8_2_std_dev', 'mfcc_8_2_min', 'mfcc_8_2_max',
    'mfcc_8_2_median', 'mfcc_8_2_skew', 'mfcc_8_2_kurtosis',
    'mfcc_8_3_mean', 'mfcc_8_3_std_dev', 'mfcc_8_3_min', 'mfcc_8_3_max', 'mfcc_8_3_median',
    'mfcc_8_3_skew', 'mfcc_8_3_kurtosis',

    'mfcc_9_1_mean', 'mfcc_9_1_std_dev', 'mfcc_9_1_min', 'mfcc_9_1_max',
    'mfcc_9_1_median', 'mfcc_9_1_skew', 'mfcc_9_1_kurtosis',
    'mfcc_9_2_mean', 'mfcc_9_2_std_dev', 'mfcc_9_2_min', 'mfcc_9_2_max',
    'mfcc_9_2_median', 'mfcc_9_2_skew', 'mfcc_9_2_kurtosis',
    'mfcc_9_3_mean', 'mfcc_9_3_std_dev', 'mfcc_9_3_min', 'mfcc_9_3_max', 'mfcc_9_3_median',
    'mfcc_9_3_skew', 'mfcc_9_3_kurtosis',

    'mfcc_10_1_mean', 'mfcc_10_1_std_dev', 'mfcc_10_1_min', 'mfcc_10_1_max',
    'mfcc_10_1_median', 'mfcc_10_1_skew', 'mfcc_10_1_kurtosis',
    'mfcc_10_2_mean', 'mfcc_10_2_std_dev', 'mfcc_10_2_min', 'mfcc_10_2_max',
    'mfcc_10_2_median', 'mfcc_10_2_skew', 'mfcc_10_2_kurtosis',
    'mfcc_10_3_mean', 'mfcc_10_3_std_dev', 'mfcc_10_3_min', 'mfcc_10_3_max', 'mfcc_10_3_median',
    'mfcc_10_3_skew', 'mfcc_10_3_kurtosis',

    'mfcc_11_1_mean', 'mfcc_11_1_std_dev', 'mfcc_11_1_min', 'mfcc_11_1_max',
    'mfcc_11_1_median', 'mfcc_11_1_skew', 'mfcc_11_1_kurtosis',
    'mfcc_11_2_mean', 'mfcc_11_2_std_dev', 'mfcc_11_2_min', 'mfcc_11_2_max',
    'mfcc_11_2_median', 'mfcc_11_2_skew', 'mfcc_11_2_kurtosis',
    'mfcc_11_3_mean', 'mfcc_11_3_std_dev', 'mfcc_11_3_min', 'mfcc_11_3_max', 'mfcc_11_3_median',
    'mfcc_11_3_skew', 'mfcc_11_3_kurtosis',

    'mfcc_12_1_mean', 'mfcc_12_1_std_dev', 'mfcc_12_1_min', 'mfcc_12_1_max',
    'mfcc_12_1_median', 'mfcc_12_1_skew', 'mfcc_12_1_kurtosis',
    'mfcc_12_2_mean', 'mfcc_12_2_std_dev', 'mfcc_12_2_min', 'mfcc_12_2_max',
    'mfcc_12_2_median', 'mfcc_12_2_skew', 'mfcc_12_2_kurtosis',
    'mfcc_12_3_mean', 'mfcc_12_3_std_dev', 'mfcc_12_3_min', 'mfcc_12_3_max', 'mfcc_12_3_median',
    'mfcc_12_3_skew', 'mfcc_12_3_kurtosis',

    'mfcc_13_1_mean', 'mfcc_13_1_std_dev', 'mfcc_13_1_min', 'mfcc_13_1_max',
    'mfcc_13_1_median', 'mfcc_13_1_skew', 'mfcc_13_1_kurtosis',
    'mfcc_13_2_mean', 'mfcc_13_2_std_dev', 'mfcc_13_2_min', 'mfcc_13_2_max',
    'mfcc_13_2_median', 'mfcc_13_2_skew', 'mfcc_13_2_kurtosis',
    'mfcc_13_3_mean', 'mfcc_13_3_std_dev', 'mfcc_13_3_min', 'mfcc_13_3_max', 'mfcc_13_3_median',
    'mfcc_13_3_skew', 'mfcc_13_3_kurtosis',

    'mfcc_14_1_mean', 'mfcc_14_1_std_dev', 'mfcc_14_1_min', 'mfcc_14_1_max',
    'mfcc_14_1_median', 'mfcc_14_1_skew', 'mfcc_14_1_kurtosis',
    'mfcc_14_2_mean', 'mfcc_14_2_std_dev', 'mfcc_14_2_min', 'mfcc_14_2_max',
    'mfcc_14_2_median', 'mfcc_14_2_skew', 'mfcc_14_2_kurtosis',
    'mfcc_14_3_mean', 'mfcc_14_3_std_dev', 'mfcc_14_3_min', 'mfcc_14_3_max', 'mfcc_14_3_median',
    'mfcc_14_3_skew', 'mfcc_14_3_kurtosis',

    'mfcc_15_1_mean', 'mfcc_15_1_std_dev', 'mfcc_15_1_min', 'mfcc_15_1_max',
    'mfcc_15_1_median', 'mfcc_15_1_skew', 'mfcc_15_1_kurtosis',
    'mfcc_15_2_mean', 'mfcc_15_2_std_dev', 'mfcc_15_2_min', 'mfcc_15_2_max',
    'mfcc_15_2_median', 'mfcc_15_2_skew', 'mfcc_15_2_kurtosis',
    'mfcc_15_3_mean', 'mfcc_15_3_std_dev', 'mfcc_15_3_min', 'mfcc_15_3_max', 'mfcc_15_3_median',
    'mfcc_15_3_skew', 'mfcc_15_3_kurtosis',

    'mfcc_16_1_mean', 'mfcc_16_1_std_dev', 'mfcc_16_1_min', 'mfcc_16_1_max',
    'mfcc_16_1_median', 'mfcc_16_1_skew', 'mfcc_16_1_kurtosis',
    'mfcc_16_2_mean', 'mfcc_16_2_std_dev', 'mfcc_16_2_min', 'mfcc_16_2_max',
    'mfcc_16_2_median', 'mfcc_16_2_skew', 'mfcc_16_2_kurtosis',
    'mfcc_16_3_mean', 'mfcc_16_3_std_dev', 'mfcc_16_3_min', 'mfcc_16_3_max', 'mfcc_16_3_median',
    'mfcc_16_3_skew', 'mfcc_16_3_kurtosis',

    'mfcc_17_1_mean', 'mfcc_17_1_std_dev', 'mfcc_17_1_min', 'mfcc_17_1_max',
    'mfcc_17_1_median', 'mfcc_17_1_skew', 'mfcc_17_1_kurtosis',
    'mfcc_17_2_mean', 'mfcc_17_2_std_dev', 'mfcc_17_2_min', 'mfcc_17_2_max',
    'mfcc_17_2_median', 'mfcc_17_2_skew', 'mfcc_17_2_kurtosis',
    'mfcc_17_3_mean', 'mfcc_17_3_std_dev', 'mfcc_17_3_min', 'mfcc_17_3_max', 'mfcc_17_3_median',
    'mfcc_17_3_skew', 'mfcc_17_3_kurtosis',

    'mfcc_18_1_mean', 'mfcc_18_1_std_dev', 'mfcc_18_1_min', 'mfcc_18_1_max',
    'mfcc_18_1_median', 'mfcc_18_1_skew', 'mfcc_18_1_kurtosis',
    'mfcc_18_2_mean', 'mfcc_18_2_std_dev', 'mfcc_18_2_min', 'mfcc_18_2_max',
    'mfcc_18_2_median', 'mfcc_18_2_skew', 'mfcc_18_2_kurtosis',
    'mfcc_18_3_mean', 'mfcc_18_3_std_dev', 'mfcc_18_3_min', 'mfcc_18_3_max', 'mfcc_18_3_median',
    'mfcc_18_3_skew', 'mfcc_18_3_kurtosis',

    'mfcc_19_1_mean', 'mfcc_19_1_std_dev', 'mfcc_19_1_min', 'mfcc_19_1_max',
    'mfcc_19_1_median', 'mfcc_19_1_skew', 'mfcc_19_1_kurtosis',
    'mfcc_19_2_mean', 'mfcc_19_2_std_dev', 'mfcc_19_2_min', 'mfcc_19_2_max',
    'mfcc_19_2_median', 'mfcc_19_2_skew', 'mfcc_19_2_kurtosis',
    'mfcc_19_3_mean', 'mfcc_19_3_std_dev', 'mfcc_19_3_min', 'mfcc_19_3_max', 'mfcc_19_3_median',
    'mfcc_19_3_skew', 'mfcc_19_3_kurtosis',

    'mfcc_20_1_mean', 'mfcc_20_1_std_dev', 'mfcc_20_1_min', 'mfcc_20_1_max',
    'mfcc_20_1_median', 'mfcc_20_1_skew', 'mfcc_20_1_kurtosis',
    'mfcc_20_2_mean', 'mfcc_20_2_std_dev', 'mfcc_20_2_min', 'mfcc_20_2_max',
    'mfcc_20_2_median', 'mfcc_20_2_skew', 'mfcc_20_2_kurtosis',
    'mfcc_20_3_mean', 'mfcc_20_3_std_dev', 'mfcc_20_3_min', 'mfcc_20_3_max', 'mfcc_20_3_median',
    'mfcc_20_3_skew', 'mfcc_20_3_kurtosis',
    'label'
]

In [3]:
def clusterTo2(vector, algorithm):
    '''
    function that cluster given vector and calls calcStatVal function for each of them
    :param vector: feature vector to be clustered
    :param algorithm: wanted clustering algorithm
    :return: list of statistical values of each cluster of vectors
    '''

    #this actions is needed for clustering
    vector = vector.flatten()
    vector = vector.reshape(-1, 1)

    #prepares labels for chosen clustering algorithm
    if algorithm == 1:
        cluster = KMeans(n_clusters=2, random_state=0).fit(vector)
        labels = cluster.labels_
    elif algorithm == 2:
        cluster = AgglomerativeClustering(2)
        labels = cluster.fit_predict(vector)
    elif algorithm == 3:
        cluster = GaussianMixture(n_components=2).fit(vector)
        labels = cluster.predict(vector)

    vector0 = []
    vector1 = []

    #split vector by their labels
    for i in range(len(vector)):
        if labels[i] == 0:
            vector0.append(vector[i])

        elif labels[i] == 1:
            vector1.append(vector[i])

    vector0 = np.array(vector0).reshape(1, -1)
    vector1 = np.array(vector1).reshape(1, -1)

    returnVector = []

    #calls calcStatVals function for each vector
    vector0 = calcStatVals(vector0)
    vector1 = calcStatVals(vector1)

    if vector0[0] < vector1[0]:
        returnVector += vector0 + vector1
    else:
        returnVector += vector1 + vector0

    return returnVector

In [4]:
#same as above but clusters to 3
def clusterTo3(vector, algorithm):
    vector = vector.flatten()
    vector = vector.reshape(-1, 1)

    if algorithm == 1:
        cluster = KMeans(n_clusters=2, random_state=0).fit(vector)
        labels = cluster.labels_
    elif algorithm == 2:
        cluster = AgglomerativeClustering(2)
        labels = cluster.fit_predict(vector)
    elif algorithm == 3:
        cluster = GaussianMixture(n_components=2).fit(vector)
        labels = cluster.predict(vector)

    vector0 = []
    vector1 = []
    vector2 = []

    for i in range(len(vector)):
        if labels[i] == 0:
            vector0.append(vector[i])

        elif labels[i] == 1:
            vector1.append(vector[i])

        elif labels[i] == 2:
            vector2.append(vector[i])

    print("v0: ", len(vector0))
    print("v1: ", len(vector1))
    print("v2: ", len(vector2))

    vector0 = np.array(vector0).reshape(1, -1)
    vector1 = np.array(vector1).reshape(1, -1)
    vector2 = np.array(vector1).reshape(1, -1)

    returnVector = []

    vector0 = calcStatVals(vector0)
    vector1 = calcStatVals(vector1)
    vector2 = calcStatVals(vector2)

    if vector0[0] < vector1[0]:                                            #0<1
        if vector2[0] < vector0[0]:                                     #2<0<1
            returnVector += vector2 + vector0 + vector1
        elif vector2[0] < vector1[0]:                                      #0<2<1
            returnVector += vector0 + vector2 + vector1
        else:                                                           #0<1<2
            returnVector += vector0 + vector1 + vector2
    else:                                                              #1<0
        if vector2[0] < vector1[0]:                                 #2<1<0
            returnVector += vector2 + vector1 + vector0
        elif vector2[0] < vector0[0]:                               #1<2<0
            returnVector += vector1 + vector2 + vector0
        else:                                                           #1<0<2
            returnVector += vector1 + vector0 + vector2

    return returnVector

In [6]:

audios_feat = []


#loop for each wav file. extracts waveform and sends it to above functions.
#takes statistical values of clustered vectors
for cur_dir in dir_list:
    audio_files = glob(data_dir + cur_dir + '/*.wav')
    #audio_files = glob(data_dir + '/*.wav')
    for file in audio_files:
        y, sr = librosa.load(file)
        feature_vector = extractFeature(y, 2) #currently calls gmm. 1 for k-mean, 3 for hierarchical
        feature_vector.append(cur_dir)
        audios_feat.append(feature_vector)


norm_output = 'deneme_2.csv'

#writes into csv file
with open(norm_output, 'w', newline= '') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(header_2)
    csv_writer.writerows(audios_feat)

